In [1]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.4 MB/s 
     |████████████████████████████████| 5.8 MB 52.0 MB/s 
     |████████████████████████████████| 1.3 MB 48.9 MB/s 
     |████████████████████████████████| 182 kB 62.0 MB/s 
     |████████████████████████████████| 7.6 MB 61.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=ed8176f35cceb2f1f65cb2a84ded168b40ba70e614f70e5ed1f79937d2b23ae3
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd

# directory = '/content/drive/MyDrive/text/'
directory = '/content/drive/MyDrive/전은지/text/'
metadata = pd.read_json(directory + 'metadata.json', lines=True)

In [2]:
metadata['titles'] = metadata.apply(lambda x: x['title'] + '. ' + x['sub_title'], axis=1)

# Sentence Embedding

## KoBERT

In [6]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-u7xis3ul/kobert-tokenizer_283753a2e0e940d3af1cbd57ef05ec5f
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-u7xis3ul/kobert-tokenizer_283753a2e0e940d3af1cbd57ef05ec5f
  Created wheel for kobert-tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4648 sha256=81241266c9ef1498e769a91d07b3e0fda895d45037a4d93a5d109ec094009c73
  Stored in directory: /tmp/pip-ephem-wheel-cache-drv1yaid/wheels/f7/cb/29/1a737fe71e5108dc30b04ea4a990f78ed271fa537aaf3fce7c
Successfully built kobert-tokenizer


In [3]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
def encoding(model, tokenizer, sentence):
   inputs = tokenizer.batch_encode_plus([sentence], return_tensors="pt").to(device)
   model = model.to(device)
   out = model(input_ids = torch.tensor(inputs['input_ids']),
              attention_mask = torch.tensor(inputs['attention_mask']))
   return out
  # out.pooler_output.shape

In [5]:
import torch
from transformers import BertModel
from kobert_tokenizer import KoBERTTokenizer

'''
KoBERT
https://github.com/SKTBrain/KoBERT
'''

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
from tqdm import tqdm
import pickle
import os.path

count = 1

'''
실제 embedding 추출 시에는 범위를 나눠서 KoBERT_new, new2, ... 파일을 여러 개 생성하였음
'''

if os.path.exists(directory + 'doc_embedding_KoBERT.pickle'):
    with open(directory + 'doc_embedding_KoBERT.pickle', 'rb') as handle:
      doc_embedding = pickle.load(handle)
else:
  doc_embedding = {}

for id, sentence in tqdm(zip(metadata['id'], metadata['titles'])):

  if id not in doc_embedding.keys():
    doc_embedding[id] = encoding(model, tokenizer, sentence)
    count += 1

  # if count % 5100 == 0:
  if count % 1500 == 0:
    
    with open(directory + 'doc_embedding_KoBERT.pickle', 'wb') as handle:
      pickle.dump(doc_embedding, handle, protocol=pickle.HIGHEST_PROTOCOL)

    del doc_embedding
    del encoding

    import gc
    gc.collect()

    def encoding(model, tokenizer, sentence):
      inputs = tokenizer.batch_encode_plus([sentence], return_tensors="pt").to(device)
      model = model.to(device)
      out = model(input_ids = torch.tensor(inputs['input_ids']),
                  attention_mask = torch.tensor(inputs['attention_mask']))
      return out

    with open(directory + 'doc_embedding_KoBERT.pickle', 'rb') as handle:
      doc_embedding = pickle.load(handle)

    # break

with open(directory + 'doc_embedding_KoBERT.pickle', 'rb') as handle:
  doc_embedding = pickle.load(handle)

0it [00:00, ?it/s]<ipython-input-32-da5c24d8f5ff>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = model(input_ids = torch.tensor(inputs['input_ids']),
<ipython-input-32-da5c24d8f5ff>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(inputs['attention_mask']))
1499it [00:33,  5.42it/s]<ipython-input-36-f74179c09d0d>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = model(input_ids = torch.tensor(inputs['input_ids']),
<ipython-input-36-f74179c09d0d>:35: UserWarning: To copy construct from a tensor, 

### Merging Embeddings into a Single File

In [ ]:
import pickle

with open(directory + 'doc_embedding_KoBERT_final.pickle', 'rb') as handle:
  final_embedding = pickle.load(handle)

In [ ]:
'''
doc_embedding_KoBERT_new
doc_embedding_KoBERT_new2
doc_embedding_KoBERT_new3
doc_embedding_KoBERT_new4
doc_embedding_KoBERT_new5
doc_embedding_KoBERT_new6
'''

with open(directory + 'doc_embedding_KoBERT_new.pickle', 'rb') as handle:
  doc_embedding = pickle.load(handle)

In [ ]:
final_embedding = {id: emb.pooler_output.detach().cpu().numpy() for id, emb in doc_embedding.items()}

In [ ]:
new_dict = {id: emb.pooler_output.detach().cpu().numpy() for id, emb in doc_embedding.items()}

In [ ]:
final_embedding.update(new_dict)
del new_dict
del doc_embedding

In [ ]:
import pickle

with open(directory + 'doc_embedding_KoBERT_final.pickle', 'wb') as handle:
    pickle.dump(final_embedding, handle, protocol=pickle.HIGHEST_PROTOCOL)

- 오류로 인해 pickle 파일이 손상되거나 누락된 sentence가 있는 경우 아래와 같이 해당 text에 대해서만 추가적으로 embedding을 진행

In [ ]:
import pickle
with open(directory + 'doc_embedding_KoBERT_final.pickle', 'rb') as handle:
  final_embedding = pickle.load(handle)

final_embedding_id = set(final_embedding.keys())
del final_embedding

# with open(directory + 'doc_embedding_KoBERT_left.pickle', 'rb') as handle:
#   doc_embedding = pickle.load(handle)
#   left_keys = doc_embedding.keys()
# del doc_embedding

In [ ]:
from tqdm import tqdm
import pickle
import os.path
import torch

count = 1

if os.path.exists(directory + 'doc_embedding_KoBERT_left3.pickle'):
    with open(directory + 'doc_embedding_KoBERT_left3.pickle', 'rb') as handle:
      doc_embedding = pickle.load(handle)
else:
  doc_embedding = {}

for id, sentence in tqdm(zip(metadata['id'], metadata['titles'])):

  # if (id not in doc_embedding.keys()) & (id not in final_embedding_id) & (id not in left_keys) :
  if (id not in doc_embedding.keys()) & (id not in final_embedding_id):
    doc_embedding[id] = encoding(model, tokenizer, sentence)

    torch.cuda.empty_cache()

    del encoding

    def encoding(model, tokenizer, sentence):
      inputs = tokenizer.batch_encode_plus([sentence], return_tensors="pt").to(device)
      model = model.to(device)
      out = model(input_ids = torch.tensor(inputs['input_ids']),
                  attention_mask = torch.tensor(inputs['attention_mask']))
      return out

    count += 1

  # if count % 5100 == 0:
  if count % 100 == 0:
    
    with open(directory + 'doc_embedding_KoBERT_left3.pickle', 'wb') as handle:
      pickle.dump(doc_embedding, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    torch.cuda.empty_cache()

    del doc_embedding
    del encoding

    import gc
    gc.collect()

    torch.cuda.empty_cache()

    def encoding(model, tokenizer, sentence):
      inputs = tokenizer.batch_encode_plus([sentence], return_tensors="pt").to(device)
      model = model.to(device)
      out = model(input_ids = torch.tensor(inputs['input_ids']),
                  attention_mask = torch.tensor(inputs['attention_mask']))
      return out

    with open(directory + 'doc_embedding_KoBERT_left3.pickle', 'rb') as handle:
      doc_embedding = pickle.load(handle)

    # break

with open(directory + 'doc_embedding_KoBERT_left3.pickle', 'wb') as handle:
  pickle.dump(doc_embedding, handle, protocol=pickle.HIGHEST_PROTOCOL)

216823it [00:00, 733144.90it/s]<ipython-input-4-da5c24d8f5ff>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = model(input_ids = torch.tensor(inputs['input_ids']),
<ipython-input-4-da5c24d8f5ff>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(inputs['attention_mask']))
<ipython-input-26-0131953a60c4>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = model(input_ids = torch.tensor(inputs['input_ids']),
<ipython-input-26-0131953a60c4>:28: UserWarning: To copy construct from a tensor, it is recommen

- 추출한 embedding까지 포함하여 전체 sentence embedding 파일 합치기

In [ ]:
import pickle
'''
doc_embedding_KoBERT_left
doc_embedding_KoBERT_left2
doc_embedding_KoBERT_left3
'''
with open(directory + 'doc_embedding_KoBERT_final.pickle', 'rb') as handle:
  final_embedding = pickle.load(handle)

with open(directory + 'doc_embedding_KoBERT_left3.pickle', 'rb') as handle:
  doc_embedding = pickle.load(handle)

In [ ]:
new_dict = {id: emb.pooler_output.detach().cpu().numpy() for id, emb in doc_embedding.items()}

In [ ]:
final_embedding.update(new_dict)
del new_dict
del doc_embedding

In [ ]:
metadata.shape

(643104, 10)

In [ ]:
len(final_embedding.keys())

643104

In [ ]:
import pickle

with open(directory + 'doc_embedding_KoBERT_final.pickle', 'wb') as handle:
    pickle.dump(final_embedding, handle, protocol=pickle.HIGHEST_PROTOCOL)

## GPT

In [ ]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

'''
https://github.com/SKT-AI/KoGPT2
'''

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
text = metadata['titles'][0]

In [ ]:
_ = model.eval()

prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)

In [ ]:
tokens

tensor([[ 9644,  9582, 50788, 10446,  9033, 39105,  7166, 12141,  9430, 30048,
         44515, 10307,  8263, 26478]], device='cuda:0')

In [ ]:
from tqdm import tqdm

doc_embedding = {}
for id, sentence in tqdm(zip(metadata['id'], metadata['titles'])):
  doc_embedding['id'] = encoding(model, tokenizer, sentence)

0it [00:00, ?it/s]


NameError: ignored

In [ ]:
out = encoding(model, tokenizer, sentences[0])

In [ ]:
out['logits'].shape

torch.Size([1, 21, 51200])

In [ ]:
out.keys()

odict_keys(['logits', 'past_key_values'])

In [ ]:
model.transformer.wte.weight.shape

torch.Size([51200, 768])

In [ ]:
tokenizer.batch_encode_plus([sentences[0]])

{'input_ids': [[12224, 9021, 10246, 17947, 9325, 9050, 6969, 8094, 387, 42642, 27251, 43531, 389, 19526, 13400, 9325, 9712, 27531, 9791, 13513, 9292]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

- 유사한 article 찾아보기 similarity로

## KoBART

In [ ]:
'''https://github.com/SKT-AI/KoBART'''